# 데이터 불러오기


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import re

In [ ]:
financial_data = pd.read_csv("/content/drive/MyDrive/data/Financial_data.csv", sep="|")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 데이터 분류

### 간단 전처리

In [ ]:
financial_data

,ticker1,bas_dt,px_last,px_volume,pe_ratio,chg_net_1d,chg_pct_1d
0,TK_AA1 Comdty,20010517,"14,870.00",142.00,NaN,NaN,NaN
1,TK_AA1 Comdty,20010518,"14,900.00",116.00,NaN,30.00,0.20
2,TK_AA1 Comdty,20010521,"14,990.00",102.00,NaN,90.00,0.60
3,TK_AA1 Comdty,20010522,"14,950.00",704.00,NaN,-40.00,-0.27
4,TK_AA1 Comdty,20010523,"14,930.00",828.00,NaN,-20.00,-0.13
...,...,...,...,...,...,...,...
1388675,TK_ZNA1 Comdty,20200424,"15,830.00","17,726.00",NaN,-35.00,-0.22
1388676,TK_ZNA1 Comdty,20200427,"16,185.00","17,916.00",NaN,355.00,2.24
1388677,TK_ZNA1 Comdty,20200428,"16,130.00","10,742.00",NaN,-55.00,-0.34
1388678,TK_ZNA1 Comdty,20200429,"16,320.00","11,117.00",NaN,190.00,1.18


데이터 형식 변환

In [ ]:
financial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388680 entries, 0 to 1388679
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ticker1     1388680 non-null  object
 1   bas_dt      1388680 non-null  int64 
 2   px_last     1366362 non-null  object
 3   px_volume   722782 non-null   object
 4   pe_ratio    470977 non-null   object
 5   chg_net_1d  1366057 non-null  object
 6   chg_pct_1d  1218439 non-null  object
dtypes: int64(1), object(6)
memory usage: 74.2+ MB


In [ ]:
for col in ['px_last','px_volume','pe_ratio','chg_net_1d','chg_pct_1d']:
  financial_data[col] = financial_data[col].str.replace(",", "").astype("float")

In [ ]:
financial_data["bas_dt"] = financial_data["bas_dt"].astype("str")
financial_data["bas_dt"] = pd.to_datetime(financial_data["bas_dt"], format="%Y-%m-%d")

In [ ]:
financial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388680 entries, 0 to 1388679
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   ticker1     1388680 non-null  object        
 1   bas_dt      1388680 non-null  datetime64[ns]
 2   px_last     1366362 non-null  float64       
 3   px_volume   722782 non-null   float64       
 4   pe_ratio    427233 non-null   float64       
 5   chg_net_1d  978900 non-null   float64       
 6   chg_pct_1d  718701 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 74.2+ MB


### 데이터 분할

# 새 섹션

In [ ]:
ticker_info = pd.read_excel("/content/drive/MyDrive/data/Ticker_master.xlsx", sheet_name="Sheet1")
ticker_info

,category,category1,category2,category3,bb_function,bb_function_kor,ticker,name_kor,name_eng,source_name,fut_exch_name_long,country_iso,country_full_name,region_or_country,indx_freq,quote_units,indx_units,curr_generic_futures,parent_futures_name
0,World Index,World Index,NaN,Standard View,WEI,전세계주가지수,TK_INDU Index,다우존스 산업평균지수,Dow Jones Industrial Average,Dow Jones,NaN,US,NaN,North/Latin America,NaN,NaN,Points,NaN,NaN
1,World Index,World Index,NaN,Standard View,WEI,전세계주가지수,TK_SPX Index,S&P 500지수,S&P 500 Index,Standard & Poor's,NaN,US,NaN,North/Latin America,NaN,NaN,Points,NaN,NaN
2,World Index,World Index,NaN,Standard View,WEI,전세계주가지수,TK_CCMP Index,나스닥 지수,NASDAQ Composite Index,NASDAQ Global Market,NaN,US,NaN,North/Latin America,NaN,NaN,Points,NaN,NaN
3,World Index,World Index,NaN,Standard View,WEI,전세계주가지수,TK_IBOV Index,보베스파 브라질 상파울루 증권거래소 지수,Ibovespa Brasil Sao Paulo Stock Exchange Index,BOVESPA,NaN,BR,NaN,North/Latin America,NaN,NaN,Points,NaN,NaN
4,World Index,World Index,NaN,Standard View,WEI,전세계주가지수,TK_SX5E Index,유로 STOXX 50 지수,EURO STOXX 50 Price EUR,STOXX,NaN,EU,NaN,Europe/Africa,NaN,NaN,Points,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,economic,economic,한국,소매부문,ECST,전세계경제통계,TK_KOVMDOMS Index,한국 자동차 국내판매 대수,South Korea Motor Vehicle Domestic,Korea Automobile Manufacturers Association,NaN,KR,NaN,South Korea,Monthly,Number of cars,Units,NaN,NaN
305,economic,economic,한국,소매부문,ECST,전세계경제통계,TK_KOVMEXPS Index,한국 자동차 수출판매 대수,South Korea Motor Vehicle Export Sales,Korea Automobile Manufacturers Association,NaN,KR,SOUTH KOREA,South Korea,Monthly,Number of cars,Units,NaN,NaN
306,economic,economic,한국,통화부문,ECST,전세계경제통계,TK_KOFETOT Index,한국 외환보유액,South Korea Foreign Exchange International Res...,Bank of Korea,NaN,KR,NaN,South Korea,Monthly,BILLIONS,Value,NaN,NaN
307,economic,economic,한국,통화부문,ECST,전세계경제통계,TK_KOFETOT% Index,외환보유액 전년동월대비 증감률,South Korea Foreign Exchange Reserves YoY,Bank of Korea,NaN,KR,SOUTH KOREA,South Korea,Monthly,% CHANGE,Rate,NaN,NaN


In [ ]:
merged_data = pd.merge(financial_data, ticker_info, how="left", left_on="ticker1", right_on="ticker")

In [ ]:
merged_data["category"].value_counts()

MSCI           414048
MoneyMarket    371795
World Index    211412
commodity      134885
future         122348
Barclays        95435
economic        38757
Name: category, dtype: int64

In [ ]:
cat_name = merged_data["category"].value_counts().index.values
cat_list = [i.replace(" ", "_") for i in cat_name]

In [ ]:
for i in merged_data["category"].value_counts().index.values:
  vars()[i.replace(" ", "_")] = merged_data[merged_data["category"]==i]

In [ ]:
for i in cat_list:
  fname = "/content/drive/Shareddrives/ASAC_최종프로젝트/data/"+i+".csv"
  vars()[i].to_csv(fname, index=False)